<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/colab/05_Asthma_Mortality_BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔥 Burned Area (BA) Data

In this notebook, we will calculate the annual burned areas for each department from 2001 to 2022. This dataset will be aggregated and normalized by the surface area of each department and normalized by 1000 km<sup>2</sup>.


We will use the MODIS product MCD64A1.061 (MODIS Burned Area Monthly Global 500m)<sup>1</sup> available on Google Earth Engine (GEE) at https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MCD64A1

##📦 Import Required Libraries

In [1]:
# geospatial data handling
import geopandas as gpd
import geemap
import ee

# maping ansd visualization
import matplotlib.pyplot as plt
import folium
import geemap.foliumap as geemap_folium

# data frame handling
import pandas as pd

# other libraries
from typing import Union, List
from itables import init_notebook_mode

## 🌍 Connect to Google Earth Engine (GEE)

In [ ]:
# trigger the authentication flow
ee.Authenticate()

In [ ]:
# initialize the library.
ee.Initialize(project='ee-pdt')
print(ee.String('Hello from the Earth Engine servers!').getInfo())

## 🖥️ Calculating BA data per departaments

To get the BA per departments from 2001 to 2022 we will create a function to compute the normalized burned area (per 1000 km²) for a given year and return it as a FeatureCollection

In [ ]:
# change to my computer home directory
%cd work

/home/jovyan/work


### Compute the normalized burned area (per 1000 km²) from 2001 to 2022

In [ ]:
def compute_normalized_burned_area(year: Union[int, str], departments_fc: ee.FeatureCollection, country_fc: ee.FeatureCollection) -> ee.FeatureCollection:
    """
    Compute the normalized burned area (per 1000 km²) for a given year and return it as a FeatureCollection.
    """

    # Ensure the year is a string (needed for building date strings)
    year = str(year)

    # Print a message for tracking which year is being processed
    print(f"Processing year {year}...")

    # Load the MCD64A1.061 MODIS Burned Area  for the selected year, select the 'BurnDate' band,
    # and mask out unburned pixels (set values > 0 to 1)
    dataset = ee.ImageCollection('MODIS/061/MCD64A1') \
        .filterDate(f'{year}-01-01', f'{year}-12-31') \
        .select('BurnDate') \
        .map(lambda img: img.gt(0).selfMask())

    # Take the maximum burn date value per pixel (this creates a binary mask: burned = 1)
    max_ba = dataset.max()

    # Clip the burned area image to the boundaries of the country (e.g., Argentina)
    max_ba_arg = max_ba.clip(country_fc)

    # Set default CRS and scale
    # Aggregate finer-resolution pixels
    max_ba_arg_1km = max_ba_arg \
        .setDefaultProjection(crs='EPSG:4326', scale=1000) \
        .reduceResolution(reducer=ee.Reducer.max(), maxPixels=1024) \
        .reproject(crs='EPSG:4326', scale=1000)# Reproject to enforce 1 km resolution

    # Create an image where each pixel represents its area in km²
    pixel_area = ee.Image.pixelArea().divide(1e6)

    # Multiply the binary burned area mask by the pixel area to get burned area in km²
    burned_area_km2 = max_ba_arg_1km.multiply(pixel_area)

    # Define a nested function to compute burned area stats for each department
    def compute_ba(feat):
        # Use reduceRegion to sum burned area values within each department
        stats = burned_area_km2.reduceRegion(
            reducer=ee.Reducer.sum(),        # Sum of burned area
            geometry=feat.geometry(),        # Department geometry
            scale=1000,                      # Analysis resolution (1 km)
            maxPixels=1e13                   # High limit for pixel processing
        )
        # Extract total burned area in km²
        ba_km2 = ee.Number(stats.get('BurnDate'))

        # Compute the area of the department in km²
        dept_area = feat.geometry().area().divide(1e6)

        # Normalize the burned area per 1000 km² of department area
        normalized_ba = ba_km2.divide(dept_area).multiply(1000)

        # Round the result to 2 decimal places
        normalized_ba = normalized_ba.multiply(100).round().divide(100)

        # Add the normalized value as a new property (e.g., NBA_2001)
        return feat.set({f'NBA_{year}': normalized_ba})

    # Apply the function to all department features
    return departments_fc.map(compute_ba)

In [4]:
# Load input feature collections
argentina = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_poly') # Argentina boundaries
# dataset with mortality rate, Pm2.5, population and departments
departments = ee.FeatureCollection('projects/ee-pdt/assets/tma/tma_pm25_2001_2022')

In [ ]:
# Initialize an empty GeoDataFrame
combined_gdf = None

In [ ]:
# Loop over the range of years from 2001 to 2022 (inclusive)
for year in range(2001, 2023):

    # Call the function to compute normalized burned area (NBA) for the current year
    # using the departments and Argentina boundary FeatureCollections
    result_fc = compute_normalized_burned_area(year, departments, argentina)

    # Convert the resulting Earth Engine FeatureCollection to a GeoDataFrame
    result_gdf = geemap.ee_to_gdf(result_fc)

    # If this is the first year, initialize the combined GeoDataFrame
    if combined_gdf is None:
        combined_gdf = result_gdf
    else:
        # For subsequent years, merge the new year’s NBA column into the combined GeoDataFrame
        # We keep only the geometry and NBA column for merging
        combined_gdf = combined_gdf.merge(
            result_gdf[[result_gdf.geometry.name, f'NBA_{year}']],  # keep geometry and current year column
            on=combined_gdf.geometry.name  # merge based on geometry
        )


In [ ]:
# Display the first few rows of the DataFrame
init_notebook_mode(all_interactive=True)
combined_gdf.head()

In [ ]:
# Save the GeoDataFrame to a GeoPackage file
combined_gdf.to_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_2001_2022.gpkg", driver="GPKG")

### Visualize statistics of NBA columns

In [ ]:
# Create a summary table of statistics for all NBA columns
nba_columns = [col for col in combined_gdf.columns if 'NBA_' in col]
summary_stats = combined_gdf[nba_columns].describe()

# Display the summary table
init_notebook_mode(all_interactive=True)
summary_stats

### Line chart of mean NBA across the years

In [ ]:
# Extract the years and mean NBA values from the summary statistics
years = [int(col.replace('NBA_', '')) for col in summary_stats.columns]
mean_nba = summary_stats.loc['mean'].values

# Create the line chart
plt.figure(figsize=(12, 6))
plt.plot(years, mean_nba, marker='o')
plt.xlabel('Year')
plt.ylabel('Mean NBA')
plt.title('Mean NBA (Normalized Burned Area) 2001-2022')
plt.grid(True)
plt.show()

### Map NBA and BA for 2003

According to our line chart, the largest normalized burned area (NBA) occurred in 2003. Therefore, we will create a map displaying both burned areas (BA) and normalized burned areas (NBA) for that year.

In [ ]:
# Load MCD64A1.061 burned area data for 2003
dataset = ee.ImageCollection('MODIS/061/MCD64A1') \
    .filterDate('2003-01-01', '2003-12-31') \
    .select('BurnDate')

# Binary burned area (1 = burned, 0 = not burned)
dataset = dataset.map(lambda img: img.gt(0).selfMask())

# Max burn date per pixel
max_ba = dataset.max()

# Load Argentina polygon
argentina = ee.FeatureCollection('projects/ee-pdt/assets/argentina/ar_poly')

# Clip image to Argentina
max_ba_arg = max_ba.clip(argentina)

# Reduce resolution 1 km and set a valid default projection (MODIS-like: 1 km in EPSG:4326)
max_ba_arg_1km = max_ba_arg \
    .setDefaultProjection(crs='EPSG:4326', scale=1000) \
    .reduceResolution(reducer=ee.Reducer.max(), maxPixels=1024) \
    .reproject(crs='EPSG:4326', scale=1000)

In [ ]:
# Visualization for burned area
ba_vis = {
    'min': 0,
    'max': 1,
    'palette': ['ffffff', 'ff0000']
}
map_id_dict = ee.Image(max_ba_arg_1km).getMapId(ba_vis)

# Load NBA data from GeoPackage (local)
# gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/tma_pm25_ba_2001_2022.gpkg")
# use in-memory gdf
gdf = combined_gdf
nba_col = "NBA_2003"

# Ensure GeoDataFrame is in EPSG:4326 for Folium
if gdf.crs != "EPSG:4326":
    gdf = gdf.to_crs("EPSG:4326")

# Create Folium map centered on Argentina
m = folium.Map(location=[-38.4, -63.6], zoom_start=4)

# Add burned area from EE
folium.TileLayer(
    tiles=map_id_dict['tile_fetcher'].url_format,
    attr='Map Data © Google Earth Engine',
    name='Burned Area 2003 - Argentina',
    overlay=True,
    control=True
).add_to(m)

# Add NBA choropleth layer
folium.Choropleth(
    geo_data=gdf,
    name="Normalized Burned Area 2003",
    data=gdf,
    columns=[gdf.index, nba_col],
    key_on="feature.id",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.4,
    nan_fill_color="gray",
    legend_name="Normalized Burned Area (per 1000 km²)"
).add_to(m)

# Add boundary
argentina_geojson = geemap.ee_to_geojson(argentina)
folium.GeoJson(
    argentina_geojson,
    name='Argentina Boundary',
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'black',
        'weight': 2
    }
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Show the map
m

## 📚 References

1. Giglio, L., Justice, C., Boschetti, L., Roy, D. (2021). <i>MODIS/Terra+Aqua Burned Area Monthly L3 Global 500m SIN Grid V061</i> [Data set]. NASA EOSDIS Land Processes Distributed Active Archive Center. Accessed 2025-05-10 from https://doi.org/10.5067/MODIS/MCD64A1.061
